In [34]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import random
from rank_bm25 import BM25Okapi
import ast

In [68]:
train_or_test = "train"
save_name = "MC_MedQA_textbooks_train"

## Load MedQA textbooks

In [3]:
source = "/scratch/s190619/Data_etc/MedQA/textbooks/combined_text_books.txt"

textbooks = []
f = open(source, "r")
for x in f:
    textbooks.append(x)
f.close()

TB = [x.replace("\n","") for x in textbooks if x != "\n"]
TB_l = [x.lower() for x in TB]

## Load MedQA 

In [18]:
queries_train.to_csv("/scratch/s190619/Data_etc/MedQA/disorders_table_train.csv",sep=",",index=False)

In [19]:
#queries = pd.read_csv("/scratch/s190619/MSMARCO/MedQA_US/MedQAQueries_disorders_full.csv",sep=",")
#queries_train = pd.read_csv("/scratch/s190619/Data_etc/MedQA/MedQAQueries_disorders_full_v2.csv",sep=",").drop("Unnamed: 0",axis=1)
queries_train = pd.read_csv("/scratch/s190619/Data_etc/MedQA/disorders_table_train.csv",sep=",")
queries_test = pd.read_csv("/scratch/s190619/Data_etc/MedQA/disorders_table_dev-test.csv",sep=",")
queries_train.head(5)

,qid,query,answer,options,meta_info,Disorder,CUI,TUI,short_category,long_category,description,cui_METAMAP
0,3,A pulmonary autopsy specimen from a 58-year-ol...,Thromboembolism,"{'A': 'Thromboembolism', 'B': 'Pulmonary ische...",step1,True,C0040038,T046,DISO,Disorders,Pathologic Function,['C0040038']
1,4,A 20-year-old woman presents with menorrhagia ...,Von Willebrand disease,"{'A': 'Factor V Leiden', 'B': 'Hemophilia A', ...",step1,True,C0042974,T047,DISO,Disorders,Disease or Syndrome,['C0042974']
2,5,A 40-year-old zookeeper presents to the emerge...,Scorpion sting,"{'A': 'Aspirin', 'B': 'Oral contraceptive pill...",step1,True,C0238417,T037,DISO,Disorders,Injury or Poisoning,['C0238417']
3,5,A 40-year-old zookeeper presents to the emerge...,Scorpion sting,"{'A': 'Aspirin', 'B': 'Oral contraceptive pill...",step1,True,C0261663,T037,DISO,Disorders,Injury or Poisoning,['C0238417']
4,21,An investigator is studying the function of th...,Anorexia,"{'A': 'Hypothermia', 'B': 'Hyperthermia', 'C':...",step1,True,C0003123,T047,DISO,Disorders,Disease or Syndrome,"['C0003123', 'C1971624']"


## Create triples

In [24]:
def filter_textbook_by_answers(queries, passages):
    
    qa_pairs = queries[["query","answer"]]
    number_of_unrelated_passages = 10
    
    relevant_passages = []
    unrelevant_passages = []
    for q,a in tqdm(qa_pairs.to_numpy()):
        a_l = a.lower()
        un_pa = []
        try: 
            int(a_l)
            relevant_passages.append([])
            unrelevant_passages.append([])
        except: 
            if len(a_l) >= 4: #The answer to some questions is simply A, B, C, D, E, or some number
                relevant_passages.append([x.replace("\t"," ") for x in passages if a_l in x])
                
                # Get unrelevant passages
                k=0
                while k < number_of_unrelated_passages:
                    ind = random.randint(0,len(passages))
                    if a_l.lower() not in passages[ind] and len(passages[ind]) > 250 and len(passages[ind]) < 2000:
                        un_pa.append(passages[ind].replace("\t"," "))
                        k += 1
                unrelevant_passages.append(un_pa) 
                
            else:
                relevant_passages.append([])
                unrelevant_passages.append([])
    
    qa_pairs["relevant_passages"] = relevant_passages
    qa_pairs["unrelevant_passages"] = unrelevant_passages
    return qa_pairs

In [30]:
%%time
if train_or_test == "train":
    queries = queries_train
    
qa_2 = filter_textbook_by_answers(queries, TB_l)
qa_2 = qa_2.drop_duplicates(subset="query").reset_index().drop("index", axis=1)
a = len([x for x in qa_2["relevant_passages"] if x==[]])
b = len(qa_2["relevant_passages"])
print("Percentage of questions covered: %.2f percent" % ((1-a/b)*100))
qa_2.head()

  0%|          | 0/2616 [00:00<?, ?it/s]

Percentage of questions covered: 93.60 percent
CPU times: user 3min 36s, sys: 4.03 s, total: 3min 40s
Wall time: 3min 39s


/home/s190619/.conda/envs/colbert-v0.2/lib/python3.7/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/s190619/.conda/envs/colbert-v0.2/lib/python3.7/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,query,answer,relevant_passages,unrelevant_passages
0,A pulmonary autopsy specimen from a 58-year-ol...,Thromboembolism,"[direct thrombin bivalirudin, argatroban, dabi...",[somewhat similar results were published from ...
1,A 20-year-old woman presents with menorrhagia ...,Von Willebrand disease,[adhesion of platelets to exposed collagen at ...,[in the postganglionic type of autonomic failu...
2,A 40-year-old zookeeper presents to the emerge...,Scorpion sting,"[causes: idiopathic, gallstones, ethanol, trau...",[ortho-phthalaldehyde (opa) is a phenolic dial...
3,An investigator is studying the function of th...,Anorexia,[kwashiorkor: kwashiorkor occurs when protein ...,[lugaro cells have fusiform somata from which ...
4,A 42-year-old woman is in the hospital recover...,Delirium tremens,[delirium is a clinical diagnosis that is made...,[intellectual disability (intellectual develop...


In [31]:
def BM25_relevant_passages(qap):
    #qap = question answer passages
    max_doc_len = 2000 # Some documents are really long, we do not want those
#     max_tokens = 180
    qs = []
    ans = []
    rel_pas = []
    unrel_pas = []
    
    for index in tqdm(range(qap.shape[0])):
    
        query = qap["query"].iloc[index].lower()
        answer = qap["answer"].iloc[index]
        corpus = [doc for doc in qap["relevant_passages"].iloc[index] if len(doc)<max_doc_len]
        
        if len(corpus) > 0:
            tokenized_corpus = [doc.lower().split(" ") for doc in corpus]
            bm25 = BM25Okapi(tokenized_corpus)
            tokenized_query = query.split(" ")
            doc_scores = bm25.get_scores(tokenized_query)
            N = min(len(doc_scores[np.where(doc_scores>doc_scores.max()/2)]),10)
            relevant_passages = bm25.get_top_n(tokenized_query, corpus, n=N)
            
            #tokenize_passages = [doc.lower().split(" ")[:max_tokens] for doc in relevant_passages]
            #relevant_passages = [" ".join([x for x in doc]) for doc in tokenized_passages]
            
            qs.append(query)
            ans.append(answer)
            rel_pas.append(relevant_passages)
            unrel_pas.append(random.sample(qap["unrelevant_passages"].iloc[index],len(relevant_passages)))
        
    return pd.DataFrame(list(zip(qs,ans,rel_pas,unrel_pas)), columns = ["query","answer","relevant_passages","unrelevant_passages"])

In [32]:
qa_2_bm25 = BM25_relevant_passages(qap = qa_2)
qa_2_bm25.head(1)

qs = []
ans = []
rel_pas = []
unrel_pas = []
for i in tqdm(range(qa_2_bm25.shape[0])):
    q = qa_2_bm25.loc[i,"query"]
    a = qa_2_bm25.loc[i,"answer"]
    tmp_rel_pas = qa_2_bm25.loc[i,"relevant_passages"]
    tmp_unrel_pas = qa_2_bm25.loc[i,"unrelevant_passages"]
    for i in range(len(tmp_rel_pas)):
        qs.append(q)
        ans.append(a)
        rel_pas.append(tmp_rel_pas[i])
        unrel_pas.append(tmp_unrel_pas[i])
        
qa_long = pd.DataFrame(list(zip(qs,ans,rel_pas,unrel_pas)), columns = ["query","answer","relevant_passage","unrelevant_passage"])
qa_long = qa_long.sample(frac=1).reset_index(drop=True)
print("Shape: ", qa_long.shape)
qa_long.head()

  0%|          | 0/2281 [00:00<?, ?it/s]

  0%|          | 0/2121 [00:00<?, ?it/s]

Shape:  (16645, 4)


,query,answer,relevant_passage,unrelevant_passage
0,a 66-year-old man comes to the physician for a...,Acute myelogenous leukemia,ataxia telangiectasia causes large telangiecta...,another approach to reducing cocaine abuse is ...
1,a 32-year-old man presents with hypertension t...,Primary aldosteronism,several sporadic and familial adrenal abnormal...,other indoor exposures of concern are bioaeros...
2,a 72-year-old man is brought to the physician ...,Vascular dementia,experienced neurologists recognize that certai...,many of the processes that aid cervical remode...
3,a 7-year-old girl is brought to the physician ...,Granulosa cell tumor,surgery because granulosa cell tumors are bila...,"embryologically, the functions of the ear—hear..."
4,a 29-year-old primigravid woman at 24 weeks' g...,Hypocalcemia,a number of nonhereditary metabolic diseases m...,"in a series of experiments with monkeys, ranso..."


In [48]:
options = []
for answer in tqdm(qa_long["answer"]):
    opt = queries[queries["answer"]==answer]["options"].iloc[0]
    options.append([x for x in ast.literal_eval(opt).values() if x != answer])
    
qa_long["options1"] = [x[0] for x in options]
qa_long["options2"] = [x[1] for x in options]
qa_long["options3"] = [x[2] for x in options]
qa_long["options4"] = [x[3] for x in options]

  0%|          | 0/16645 [00:00<?, ?it/s]

In [52]:
qa_long.head()

,query,answer,relevant_passage,unrelevant_passage,options1,options2,options3,options4
0,a 66-year-old man comes to the physician for a...,Acute myelogenous leukemia,ataxia telangiectasia causes large telangiecta...,another approach to reducing cocaine abuse is ...,Myelodysplastic syndrome,Acute lymphoblastic leukemia,Chronic myelogenous leukemia,Hairy cell leukemia
1,a 32-year-old man presents with hypertension t...,Primary aldosteronism,several sporadic and familial adrenal abnormal...,other indoor exposures of concern are bioaeros...,Cushing syndrome,Essential hypertension,Coarctation of aorta,Renal artery stenosis
2,a 72-year-old man is brought to the physician ...,Vascular dementia,experienced neurologists recognize that certai...,many of the processes that aid cervical remode...,Normal aging,Alzheimer's dementia,Lewy body dementia,Presbycusis
3,a 7-year-old girl is brought to the physician ...,Granulosa cell tumor,surgery because granulosa cell tumors are bila...,"embryologically, the functions of the ear—hear...",Congenital adrenal hyperplasia,Idiopathic precocious puberty,McCune-Albright syndrome,Sertoli-Leydig tumor
4,a 29-year-old primigravid woman at 24 weeks' g...,Hypocalcemia,a number of nonhereditary metabolic diseases m...,"in a series of experiments with monkeys, ranso...",Omphalocele,Intrauterine growth restriction,Hypermagnesemia,Hyperglycemia


In [64]:
dfs = []

df_tmp = qa_long[["relevant_passage","unrelevant_passage","answer"]]
df_tmp["query"] = qa_long["query"] + [' ' for _ in range(qa_long.shape[0])] + qa_long["answer"] 
df_tmp = df_tmp[["query","relevant_passage","unrelevant_passage","answer"]]
dfs.append(df_tmp)

for i in range(1,5):
    df_tmp = qa_long[["relevant_passage","unrelevant_passage","answer"]]
    df_tmp["query"] = qa_long["query"] + [' ' for _ in range(qa_long.shape[0])] + qa_long[f"options{i}"] 
    df_tmp = df_tmp[["query","relevant_passage","unrelevant_passage","answer"]]
    dfs.append(df_tmp)

/home/s190619/.conda/envs/colbert-v0.2/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/s190619/.conda/envs/colbert-v0.2/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [72]:
import os
save_path = "/scratch/s190619/Data_etc/ColBERT/training_sets/" + save_name + "/"
if not os.path.exists(save_path):
    os.mkdir(save_path)

In [75]:
file_names = ["w_answer_correct_answer.tsv",
              "w_answer_option1.tsv",
              "w_answer_option2.tsv",
              "w_answer_option3.tsv",
              "w_answer_option4.tsv"]
for i in range(5):
    dfs[i].to_csv(save_path+file_names[i],sep="\t")
    
file_names = ["correct_answer.tsv",
              "option1.tsv",
              "option2.tsv",
              "option3.tsv",
              "option4.tsv"]
for i in range(5):
    dfs[i].drop("answer",axis=1).to_csv(save_path+file_names[i],sep="\t",index=False,header=False)

In [77]:
#test it
# df_test = pd.read_csv(save_path+file_names[0],sep="\t")
# df_test.head()

In [40]:
mydict = {'A' : "Hello", 'B' : "World"}